In [87]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import math

sp500 = pd.read_csv("MULTPL-SP500_REAL_PRICE_MONTH.csv", parse_dates = ['Date'])
sp500 = sp500[sp500['Date'] >= '1920-01-01'].sort_values(by=['Date'])
max_index = sp500.index.max()
sp500.index = max_index - sp500.index

sp500["Date"] = pd.to_datetime(sp500["Date"])

min_x  = min(sp500["Date"])
min_y  = 1
max_y  = 5000
sp500fig = px.line(sp500, x="Date", y="Value", title='S&P 500 Value', log_y=True, range_y=[min_y, max_y])

def add_events(figure, min_x, min_y, max_y):
    events = [{ 'descr': 'Great depression',
                'start': '1929-09-01',
                'end'  : '1933-03-01'
              },
              { 'descr': 'Dotcom bubble',
                'start': '2000-03-01',
                'end'  : '2002-10-01'
              },
              { 'descr': 'Global financial crisis',
                'start': '2007-12-01',
                'end'  : '2009-06-01'
              },
             ]

    for event in events:
        if pd.to_datetime(event['end']) >= min_x:
            figure.add_shape(
                type="rect",
                x0=event['start'], y0=min_y, x1=event['end'], y1=max_y,
                line=dict( width=1),
            )
            figure.add_trace(
                go.Scatter(
                    x=[event['start'], event['start'], event['end'], event['end'], event['start']], 
                    y=[min_y         , max_y         , max_y       , min_y       , min_y],
                    fill="toself",
                    mode='lines',
                    name='',
                    text=event['descr'],
                    opacity=0
                )
            )

add_events(figure=sp500fig, min_x=min_x, min_y=min_y, max_y=max_y)
sp500fig.show()

In [88]:
deposit_per_month = 500
def simulate_cost_average_starting(inv_years, month_start):
    inv_hist = sp500[month_start: month_start+inv_years*12].copy()
    inv_hist['Units'] = deposit_per_month / inv_hist["Value"]
    inv_end_date = inv_hist['Date'].iat[-1]
    final_unit_value = inv_hist['Units'].sum() * inv_hist["Value"].iat[-1]
    inv_return = final_unit_value / ( deposit_per_month * inv_years*12 ) - 1
    return inv_return, inv_end_date

def simulate_cost_average(inv_years):
    inv_sim = sp500.index[:-inv_years*12].map(lambda month: simulate_cost_average_starting(inv_years, month)).tolist()
    inv_sim = pd.DataFrame(inv_sim, columns=['Return over total deposits','Redemption date'])
    return inv_sim

def plot_cost_average(inv_years):
    inv_sim = simulate_cost_average(inv_years)
    figure = px.line(
        inv_sim, 
        x="Redemption date", 
        y="Return over total deposits", 
        title=str(inv_years) + ' year investment return'
    )
    figure.update_layout(yaxis=dict(tickformat=".2%"))
    min_x=min(inv_sim['Redemption date'])
    min_y=math.ceil(min(inv_sim['Return over total deposits']))  - 1
    max_y=math.floor(max(inv_sim['Return over total deposits'])) + 1
    add_events(figure=figure, min_x=min_x, min_y=min_y, max_y=max_y)
    figure.show()
    
inv_horizons_years = [10, 20, 30]
for inv_years in inv_horizons_years:
    plot_cost_average(inv_years)